# Units, Quantities, and Constants

The [astropy.units](https://docs.astropy.org/en/stable/units/) sub-package in astropy provides a way to define units and quantities (values or arrays with a unit attached), as well as a way to convert between them. It supports advanced features such as equivalencies to convert between units that are not strictly equivalent (e.g. various spectral coordinates) as well as e.g. logarithmic units.


<section class="objectives panel panel-warning">
<div class="panel-heading">
<h2><span class="fa fa-certificate"></span> Objectives</h2>
</div>


<div class="panel-body">

<ul>
<li>Access units and create quantities</li>
<li>Combine quantities together</li>
<li>Convert quantities to different units</li>
<li>Understand and use equivalencies</li>
<li>Use logarithmic units</li>
<li>Plot quantities with units</li>
</ul>

</div>

</section>


## Documentation

This notebook only shows a subset of the functionality in astropy.units. For more information about the features presented below as well as other available features, you can read the
[astropy.units documentation](https://docs.astropy.org/en/stable/units/).

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
plt.rc('image', origin='lower')
plt.rc('figure', figsize=(10, 6))

## Representing units and quantities

Astropy includes a framework that allows users to attach units to scalars and arrays, and manipulate/combine these, keeping track of the units.

Since we may want to use a number of units in expressions, it is easiest and most concise to import the units module with:

In [ ]:
import astropy.units as u

Units can then be accessed with:

In [ ]:
u.m

In [ ]:
u.pc

In [ ]:
u.s

In [ ]:
u.kg

We can also create composite units:

In [ ]:
u.m / u.kg / u.s**2

The most useful feature about the units is the ability to attach them to scalars or arrays, creating ``Quantity`` objects:

In [ ]:
time = 3. * u.Myr
time

In [ ]:
weights = [4, 5, 6] * u.kg
weights

In [ ]:
import numpy as np
distance = np.array([1.2, 2.2, 1.7]) * u.kpc
distance

and we can use this with composite units:

In [ ]:
H0 = 75 * u.km / u.s / u.Mpc
H0

We can access the unit and the value of a quantity separately:

In [ ]:
H0.value

In [ ]:
H0.unit

Finally, note that 'imperial' units can be accessed using:

In [ ]:
from astropy.units import imperial
imperial.gallon

## Combining and converting quantities

Quantities can be combined via standard arithmetic - for example multiplication and division will result in the units being combined:

In [ ]:
speed = distance / time
speed

while addition and subtraction will work only if the units are compatible:

In [ ]:
start_distance = 10 * u.kpc
end_distance = 0.3 * u.Mpc
end_distance - start_distance

Quantities can be converted to different units using ``.to``:

In [ ]:
speed.to(u.km / u.s)

In [ ]:
speed.to(u.au / u.year)

There are also convenience attributes to convert quantities to e.g. SI and cgs units:

In [ ]:
speed.si

In [ ]:
speed.cgs

Finally, by default units will not be simplified - but you can use the ``.decompose()`` method for this:

In [ ]:
speed / H0

In [ ]:
(speed / H0).decompose()

or you can convert directly to the desired unit:

In [ ]:
(speed / H0).to(u.Mpc)


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge: Unit conversions</h2>
</div>


<div class="panel-body">

<ol>
<li>Convert the speed above to miles/hour</li>
<li>Calculate whether a pint is more than half litre.
    <em>You can compare quantities as comparing variables.</em>
    Notice something strange? Check what deffinition of <a href="https://en.wikipedia.org/wiki/Pint">pint</a> astropy is using.</li>
<li>Calculate the area of a rectangle of 3 km of side and 5 meter of width. Show this in $m^2$ and also convert it to yards$^2$</li>
</ol>

</div>

</section>


In [ ]:
#1
from astropy.units import imperial
print(speed.to(imperial.mile/u.hour))

#2
print(imperial.pint > 0.5 * u.l)
print(imperial.pint.to(u.ml))
 # A liquid pint in US is 473 ml; in UK is 568 ml

#3
rectangle_area = 3 * u.km * 5 * u.m
print(rectangle_area)
print(rectangle_area.decompose())
print(rectangle_area.to(imperial.yard ** 2))

## Composed units

Many units are compositions of others, and you can create new combinations for ease of use:

In [ ]:
cms = u.cm / u.s
speed.to(cms)

In [ ]:
mph = imperial.mile / u.hour
speed.to(mph)

You can use the ``compose()`` unit to find other units that are equivalent:

In [ ]:
u.joule.compose()

In [ ]:
(u.s ** -1).compose()

You can also use ``find_equivalent_units`` to print a table with more information:

In [ ]:
(u.s ** -1).find_equivalent_units()

## Dimensionless quantities

Sometime we get quantities which are effectively dimensionless:

In [ ]:
dimensionless_quantity = 20. * u.cm / (1. * u.m)
dimensionless_quantity

What happen if we add a number to this?

In [ ]:
dimensionless_quantity + 3

We can also convert to the actual dimensionless quantity using:

In [ ]:
dimensionless_quantity.decompose()

## Equivalencies

Some conversions are not done by a conversion factor as between miles and kilometers – for example converting between wavelength and frequency:

In [ ]:
(656.281 * u.nm).to(u.Hz)  # Fails because they are not compatible

However we can make use of a spectral *equivalency* to indicate the link between the units:

In [ ]:
(656.281 * u.nm).to(u.Hz, equivalencies=u.spectral())

Other built-in equivalencies are: 
 - [Parallax](https://docs.astropy.org/en/stable/units/equivalencies.html#how-to-convert-parallax-to-distance)
 - [Doppler](https://docs.astropy.org/en/stable/units/equivalencies.html#spectral-doppler-equivalencies)
 - [Spectral flux density](https://docs.astropy.org/en/stable/units/equivalencies.html#spectral-flux-luminosity-density-units)
 - [Brigthness temperature](https://docs.astropy.org/en/stable/units/equivalencies.html#brightness-temperature-surface-brightness-equivalency)
 - [Temperature energy](https://docs.astropy.org/en/stable/units/equivalencies.html#temperature-energy-equivalency)
 - and you can [build your own](http://astropy.readthedocs.org/en/stable/units/equivalencies.html#writing-new-equivalencies)

We can use ``find_equivalent_units`` as above to show how using an equivalency expands the number of equivalent units:

In [ ]:
u.Hz.find_equivalent_units()

In [ ]:
u.Hz.find_equivalent_units(equivalencies=u.spectral())


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Challenge: Using equivalencies</h2>
</div>


<div class="panel-body">

<ol>
<li>Find out more about the spectral flux equivalency and convert 12 mJy to ergs/cm^2/s/Hz and to W/m^2/Hz</li>
</ol>

</div>

</section>


In [ ]:
flux_density = 12 * u.mJy
print(flux_density.to(u.erg / u.cm ** 2 / u.s / u.Hz))
print(flux_density.to(u.W / u.m ** 2 / u.Hz))

## Logarithmic units

Quantities can also be constructed with [Logarithmic Units](https://docs.astropy.org/en/stable/units/logarithmic_units.html), including various kinds of magnitude and dex:

In [ ]:
(5. * u.STmag).to(u.erg/u.s/u.cm**2/u.AA)  

In [ ]:
(15. * u.STmag).to(u.mJy, u.spectral_density(500 * u.AA))

In [ ]:
(5. * u.ABmag).to(u.mJy)  

In [ ]:
(3 * u.dex(u.m / u.s)).to(u.cm / u.s)

## Printing the quantities

If you want to print quantities using formatting, you can use ``.value`` and ``.unit`` as shown here:

In [ ]:
print("{0.value:0.03f} {0.unit:fits}".format(time))

In [ ]:
print("{0.value:0.03f} {0.unit:latex_inline}".format(time))

## Using quantities with Numpy functions

Quantities work as expected with a number of Numpy functions (though not all):

In [ ]:
angle = np.linspace(0, 360, 1000) * u.deg
np.cos(angle)

In [ ]:
np.exp(-4 * u.m / (400 * u.cm))

## Performance considerations

If performance is important, and you are attaching units to arrays, you may want to consider using the ``<<`` operator to attach units to arrays - this prevents the array data from being copied:

In [ ]:
array = np.random.random(10000000)

In [ ]:
%timeit array * u.m / u.s

In [ ]:
%timeit array << u.m / u.s

If you are using composite units in your code, it can also be beneficial to compute the composite units once and for all as shown earlier, e.g.:

In [ ]:
kms = u.m / u.s

Finally, if having Quantity classes is still slowing down your code, you can consider using the units only to find scalefactors and then use then to modify arrays:

In [ ]:
kms_to_cgs = (u.m / u.s).to(u.cm / u.s)

In [ ]:
kms_to_cgs

## Plotting quantities

To plot quantities nicely with Matplotlib we need to use the ``quantity_support`` function:

In [ ]:
from astropy.visualization import quantity_support
quantity_support()

In [ ]:
plt.plot(angle, np.sin(angle))
plt.plot(angle.to(u.rad), np.cos(angle))

## Physical constants

The [astropy.constants](https://docs.astropy.org/en/stable/constants/index.html) sub-package provides a set of physical constants which are compatible with the units/quantities framework:

In [ ]:
from astropy.constants import M_sun, c

In [ ]:
E = 3 * M_sun * c ** 2
E.to(u.J)


<section class="challenge panel panel-success">
<div class="panel-heading">
<h2><span class="fa fa-pencil"></span> Plotting the solar system potential</h2>
</div>


<div class="panel-body">

<ol>
<li>The gravitational potential around a point source is $V=-GM/r$ where $M$ is the mass of the point and $r$ is the radius from it. Use what we've seen above to make a plot of the gravitational potential (in MJ/kg) in the solar system between 1 and 50 au.</li>
</ol>

</div>

</section>


In [ ]:
from astropy.constants import M_sun, G
r = np.linspace(1, 50, 1000) * u.au
V = (-G * M_sun / r).to(u.MJ / u.kg)
plt.plot(r, V)

<center><i>This notebook was written by <a href="https://aperiosoftware.com/">Aperio Software Ltd.</a> &copy; 2019, and is licensed under a <a href="https://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License (CC BY 4.0)</a></i></center>

![cc](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by.svg)